In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import pandas as pd
import numpy as np
import dmc
from process import processed_data

Tensorflow not installed


In [4]:
data = processed_data()
train_ids, test_ids = dmc.loading.load_ids('rawMirrored')
train, test = dmc.preprocessing.split_train_test(data, train_ids, test_ids)

## Evaluate and boost naive bayes on mirrored

In [ ]:
offset = len(train)
X, y = dmc.transformation.transform(data, ignore_features=['returnQuantity', 'orderID', 'orderDate', 'customerID'],
                                    binary_target=False)

In [ ]:
fts = dmc.transformation.transform_feature_matrix_ph(data, ignore_features=['returnQuantity', 'orderID', 'orderDate'])

In [ ]:
from dmc.classifiers import NaiveBayes, NaiveBayesM

In [ ]:
train = X[:offset], y[:offset]
test = X[offset:], y[offset:]

In [ ]:
# standard naive bayes
clf = NaiveBayes(train[0], train[1])
res = clf(test[0])
precision = dmc.evaluation.precision(res, test[1])
print(precision, ' using ', str(NaiveBayes))

In [ ]:
feature_evaluation = dmc.evaluation.evaluate_features_leaving_one_out(train[0], train[1], test[0], test[1],
                                                                      fts, NaiveBayes)

In [ ]:
# boosted with target set knowledge
negative_features = set(feature_evaluation.index[feature_evaluation.decrement < 0])
fts_series = pd.Series(fts).apply(lambda x: False if x in negative_features else True)
fts_mask = np.array(fts_series)
X_tr, X_cl = train[0].T[fts_mask].T, test[0].T[fts_mask].T
print('Train and Evaluate')
clf = NaiveBayes(X_tr, train[1])
prec = dmc.evaluation.precision(test[1], clf(X_cl))
print(prec)

## Use ensemble and optimize each splits for used features

In [4]:
# allocate memory
data = train_ids = test_ids = None
####
ensemble = dmc.ensemble.Ensemble(train, test)

In [5]:
for split in ensemble.splits:
    print(len(ensemble.splits[split][1]), split)

2082 unknown_articleID-unknown_customerID-unknown_voucherID-known_productGroup
14 unknown_articleID-unknown_customerID-known_voucherID-unknown_productGroup
22160 unknown_articleID-unknown_customerID-known_voucherID-known_productGroup
3239 unknown_articleID-known_customerID-unknown_voucherID-known_productGroup
12 unknown_articleID-known_customerID-known_voucherID-unknown_productGroup
62265 unknown_articleID-known_customerID-known_voucherID-known_productGroup
4189 known_articleID-unknown_customerID-unknown_voucherID-known_productGroup
42844 known_articleID-unknown_customerID-known_voucherID-known_productGroup
6496 known_articleID-known_customerID-unknown_voucherID-known_productGroup
93197 known_articleID-known_customerID-known_voucherID-known_productGroup


Print splits and names to know where to not drop customer for naive bayes.

In [7]:
splits = len(ensemble.splits)
scalers = [dmc.transformation.scale_features] * splits
ignore = ([None] * 3 + 
          [['returnQuantity', 'orderID', 'orderDate']] * 3 + 
          [None] * 2 + 
          [['returnQuantity', 'orderID', 'orderDate']] * 2)
ensemble.transform(scalers=scalers, ignore_features=ignore)

In [12]:
for k in ensemble.splits:
    Xt, yt = ensemble.splits[k]['train'][0], ensemble.splits[k]['train'][1]
    Xe, ye = ensemble.splits[k]['test'][0], ensemble.splits[k]['test'][1]
    fts, clf = ensemble.splits[k]['features'], dmc.classifiers.NaiveBayes
    feature_evaluation = dmc.evaluation.evaluate_features_leaving_one_out(Xt, yt, Xe, ye, fts, clf)
    negative_features = set(feature_evaluation.index[feature_evaluation.decrement < 0])
    fts_mask = np.array(pd.Series(fts).apply(lambda x: False if x in negative_features else True))
    Xt, Xe = Xt.T[fts_mask].T, Xe.T[fts_mask].T
    clf = clf(Xt, yt)
    prec = dmc.evaluation.precision(ye, clf(Xe))
    print(k, prec, len(ye))

unknown_articleID-unknown_customerID-unknown_voucherID-known_productGroup 0.6748318924111432 2082
unknown_articleID-unknown_customerID-known_voucherID-unknown_productGroup 0.8571428571428572 14
unknown_articleID-unknown_customerID-known_voucherID-known_productGroup 0.645216606498195 22160
unknown_articleID-known_customerID-unknown_voucherID-known_productGroup 0.6779870330348874 3239
unknown_articleID-known_customerID-known_voucherID-unknown_productGroup 0.6666666666666667 12
unknown_articleID-known_customerID-known_voucherID-known_productGroup 0.6542038063117321 62265
known_articleID-unknown_customerID-unknown_voucherID-known_productGroup 0.6116018142754834 4189
known_articleID-unknown_customerID-known_voucherID-known_productGroup 0.6478620110167118 42844
known_articleID-known_customerID-unknown_voucherID-known_productGroup 0.6608682266009852 6496
known_articleID-known_customerID-known_voucherID-known_productGroup 0.6649355665954912 93197


In [13]:
for k in ensemble.splits:
    Xt, yt = ensemble.splits[k]['train'][0], ensemble.splits[k]['train'][1]
    Xe, ye = ensemble.splits[k]['test'][0], ensemble.splits[k]['test'][1]
    fts, clf = ensemble.splits[k]['features'], dmc.classifiers.NaiveBayes
    clf = clf(Xt, yt)
    prec = dmc.evaluation.precision(ye, clf(Xe))
    print(k, prec, len(ye))

unknown_articleID-unknown_customerID-unknown_voucherID-known_productGroup 0.6805955811719501 2082
unknown_articleID-unknown_customerID-known_voucherID-unknown_productGroup 0.8571428571428572 14
unknown_articleID-unknown_customerID-known_voucherID-known_productGroup 0.6523014440433212 22160
unknown_articleID-known_customerID-unknown_voucherID-known_productGroup 0.657919110836678 3239
unknown_articleID-known_customerID-known_voucherID-unknown_productGroup 0.6666666666666667 12
unknown_articleID-known_customerID-known_voucherID-known_productGroup 0.6497550790974063 62265
known_articleID-unknown_customerID-unknown_voucherID-known_productGroup 0.6619718309859155 4189
known_articleID-unknown_customerID-known_voucherID-known_productGroup 0.6553076276724863 42844
known_articleID-known_customerID-unknown_voucherID-known_productGroup 0.6579433497536946 6496
known_articleID-known_customerID-known_voucherID-known_productGroup 0.6523600545081923 93197


In [35]:
for k in ensemble.splits:
    Xt, yt = ensemble.splits[k]['train'][0], ensemble.splits[k]['train'][1]
    Xe, ye = ensemble.splits[k]['test'][0], ensemble.splits[k]['test'][1]
    fts, clf = ensemble.splits[k]['features'], dmc.classifiers.Forest
    clf = clf(Xt, yt)
    prec = dmc.evaluation.precision(ye, clf(Xe))
    print(k, prec, len(ye))

Process ForkPoolWorker-1:
Traceback (most recent call last):
  File "/home/alex/anaconda3/lib/python3.5/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
  File "/home/alex/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/alex/anaconda3/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/alex/anaconda3/lib/python3.5/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
  File "/home/alex/anaconda3/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/alex/anaconda3/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/alex/anaconda3/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


KeyboardInterrupt: 

Result for optimized leaving one out

In [24]:
res1 = [(0.6748318924111432, 2082), (0.8571428571428572, 14), (0.645216606498195, 22160), (0.6779870330348874, 3239),
 (0.6666666666666667, 12), (0.6542038063117321, 62265), (0.6116018142754834, 4189), (0.6478620110167118, 42844), 
 (0.6608682266009852, 6496), (0.6649355665954912, 93197)]
prec1, size1 = np.array([e[0] for e in res1]), np.array([e[1] for e in res1])
np.sum(np.multiply(prec1, size1)/len(test))

0.65639032888227389

Result for unoptimized naive bayes

In [25]:
res2 = [(0.6805955811719501, 2082), (0.8571428571428572, 14), (0.6523014440433212, 22160), (0.657919110836678, 3239),
       (0.6666666666666667, 12), (0.6497550790974063, 62265), (0.6619718309859155, 4189), (0.6553076276724863, 42844),
       (0.6579433497536946, 6496), (0.6523600545081923, 93197)]
prec2, size2 = np.array([e[0] for e in res2]), np.array([e[1] for e in res2])
np.sum(np.multiply(prec2, size2)/len(test))

0.65286387199891749

Optimal results of both combined

In [33]:
precs = np.vstack([prec1, prec2])
optim_precs = np.max(precs, axis=0)
np.sum(np.multiply(optim_precs, size2)/len(test))

0.65934595641400762

In [34]:
optim_precs

array([ 0.68059558,  0.85714286,  0.65230144,  0.67798703,  0.66666667,
        0.65420381,  0.66197183,  0.65530763,  0.66086823,  0.66493557])

## Evaluate final precision

In [ ]:
classifiers = [dmc.classifiers.DecisionTree, dmc.classifiers.NaiveBayes, dmc.classifiers.Forest]
Xn = X[y > 0]
yn = y[y > 0]
for clf in classifiers:
    c = clf(Xn[:900000], yn[:900000])
    pred = c(Xn[900000:])
    print('precision', dmc.evaluation.dmc_cost_relative(pred, yn[900000:]), 'with', clf)

Test on Train:
```
precision 8.27814569536e-07 with <class 'dmc.classifiers.DecisionTree'>
precision 0.00368625827815 with <class 'dmc.classifiers.NaiveBayes'>
precision 8.27814569536e-07 with <class 'dmc.classifiers.Forest'>
```

Test on 25%, Train on 75%:
```
precision 0.00435064935065 with <class 'dmc.classifiers.DecisionTree'>
precision 0.00391233766234 with <class 'dmc.classifiers.NaiveBayes'>
precision 0.00393181818182 with <class 'dmc.classifiers.Forest'>
```

Evaluation of rule approach versus classifier approach

In [ ]:
def switch_predictor(quantity):
    if quantity == 1:
        return 1
    elif quantity == 2:
        return 1
    elif quantity == 3:
        return 2
    elif quantity == 4:
        return 2
    elif quantity == 5:
        return 3
    print('FAIL')
    return None

evalframe = test[test.returnQuantity > 0]
pred = evalframe.quantity.apply(switch_predictor)
print('precision', dmc.evaluation.dmc_cost_relative(pred, evalframe.returnQuantity))

Predicting using the above switch is the most precise way if one does not want to train a specially optimized regressor/classifier.